In [4]:
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import surprise
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Ratings that have to be predicted (items and users specified in sample_submission file)

In [5]:
from data_helpers import read_csv_sample

INPUT_PATH = "../data/sample_submission.csv"
ids = read_csv_sample(INPUT_PATH)

# SVD

In [9]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import Dataset
from surprise import Reader


# path to dataset file
file_path = "../data/data_surprise.csv"

reader = Reader(line_format='user item rating', sep=',', skip_lines=1)

data = Dataset.load_from_file(file_path, reader=reader)

print('SVD')
param_grid = {'n_epochs': [16, 17], 'lr_all': [0.001, 0.002],
              'reg_all': [0.05] , 'n_factors' : [40] , 'random_state' : [15]}

                              
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=2)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

SVD
1.0038627040655221
{'n_epochs': 17, 'lr_all': 0.002, 'reg_all': 0.05, 'n_factors': 40, 'random_state': 15}


In [ ]:
from surprise import accuracy

predictions = []

algo = gs.best_estimator['rmse']

for i in range(len(ids[0])):
    pred = algo.predict(ids[0][i], ids[1][i])
    predictions.append(round(pred.est))
    
print(len(predictions))

In [10]:
from surprise import accuracy

trainset, testset = train_test_split(data, test_size=.10)

algo = gs.best_estimator['rmse']

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9984


0.9984286740439706

### Save output for submission

In [4]:
from data_helpers import create_csv_submission

OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids, predictions, OUTPUT_PATH)
print("File submission.csv ready to be submitted !")

SyntaxError: invalid syntax (<ipython-input-4-b53775eec35b>, line 3)